### Predictive Model with SVM

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

#Load libraries for data processing
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from scipy.stats import norm

## Supervised learning.
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report

from scipy.io import loadmat 

# visualization
import seaborn as sns 
plt.style.use('fivethirtyeight')
sns.set_style("white")

plt.rcParams['figure.figsize'] = (8,4)

In [2]:
xx=loadmat('./kw_30_new.mat')

In [3]:
xx

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Oct 31 02:25:00 2023',
 '__version__': '1.0',
 '__globals__': [],
 'kw_30': array([[ 2.59791000e-01,  1.40403000e-01,  4.67550000e-02, ...,
          2.89551800e+01,  7.49638600e+01,  1.00000000e+00],
        [ 4.01619000e-01, -7.54210000e-02,  2.99705000e-01, ...,
          2.89551800e+01,  7.46740390e+01,  1.00000000e+00],
        [-1.35870000e-02,  1.70750000e-02,  5.02065000e-01, ...,
          2.89551800e+01,  7.48651980e+01,  1.00000000e+00],
        ...,
        [-2.59200000e-02, -3.48000000e-03,  1.22606076e+02, ...,
          0.00000000e+00,  7.54571740e+01,  2.00000000e+00],
        [ 2.85700000e-03,  1.91300000e-02,  1.22484660e+02, ...,
          0.00000000e+00,  7.53153460e+01,  2.00000000e+00],
        [ 1.51900000e-02,  3.76300000e-02,  1.22606076e+02, ...,
          0.00000000e+00,  7.53523450e+01,  2.00000000e+00]])}

In [4]:
xx.keys()

dict_keys(['__header__', '__version__', '__globals__', 'kw_30'])

In [5]:
df=xx['kw_30']
df2=xx['kw_30']

In [6]:
X=df[:,:8]

In [7]:
y=df2[:,-1]

In [8]:
X.shape

(6900, 8)

In [9]:
y.shape

(6900,)

In [10]:
X

array([[ 2.59791000e-01,  1.40403000e-01,  4.67550000e-02, ...,
         4.63985703e+03,  2.90498050e+01,  2.89551800e+01],
       [ 4.01619000e-01, -7.54210000e-02,  2.99705000e-01, ...,
         4.63986507e+03,  2.90024920e+01,  2.89551800e+01],
       [-1.35870000e-02,  1.70750000e-02,  5.02065000e-01, ...,
         4.63987312e+03,  2.89551800e+01,  2.89551800e+01],
       ...,
       [-2.59200000e-02, -3.48000000e-03,  1.22606076e+02, ...,
         4.67972150e+03,  0.00000000e+00,  0.00000000e+00],
       [ 2.85700000e-03,  1.91300000e-02,  1.22484660e+02, ...,
         4.67972150e+03,  0.00000000e+00,  0.00000000e+00],
       [ 1.51900000e-02,  3.76300000e-02,  1.22606076e+02, ...,
         4.67972150e+03,  0.00000000e+00,  0.00000000e+00]])

In [11]:
y

array([1., 1., 1., ..., 2., 2., 2.])

In [12]:
import collections
frequency = collections.Counter(y)
print(dict(frequency))

{1.0: 3104, 2.0: 3796}


In [13]:
from scipy import stats
stats.describe(X)

DescribeResult(nobs=6900, minmax=(array([-5.78946900e+00, -2.99008300e+00, -7.32331000e-01, -4.80317000e-01,
        4.82455416e+03,  4.63985703e+03, -2.03443300e+00, -1.79787100e+00]), array([4.65028100e+00, 2.92762500e+00, 1.31337914e+02, 1.31802073e+02,
       4.86450239e+03, 4.67972150e+03, 2.92390540e+01, 2.92390540e+01])), mean=array([-1.63508535e-02, -3.56452565e-03,  2.80650775e+01,  3.01091244e+01,
        4.84993212e+03,  4.66518658e+03,  2.08462690e+01,  2.08030502e+01]), variance=array([1.69230228e-01, 1.27518879e-01, 1.93256585e+03, 2.12591922e+03,
       1.71666062e+02, 1.70896166e+02, 1.40177372e+02, 1.39322502e+02]), skewness=array([-0.13304519, -0.05653312,  1.23703568,  1.17086311, -0.39351391,
       -0.39491259, -0.98740796, -0.98100431]), kurtosis=array([27.17274389, 14.99662726, -0.05796991, -0.28096028, -1.25619614,
       -1.25516708, -0.8638324 , -0.86621556]))

In [14]:
#transform the class labels from their original string representation (M and B) into integers
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize the  data (center around 0 and scale to remove the variance).
scaler =StandardScaler()
Xs = scaler.fit_transform(X)

In [15]:
Xs

array([[ 6.71312507e-01,  4.03189089e-01, -6.37391770e-01, ...,
        -1.93772765e+00,  6.92936764e-01,  6.90703931e-01],
       [ 1.01610236e+00, -2.01238066e-01, -6.31637384e-01, ...,
        -1.93711236e+00,  6.88940326e-01,  6.90703931e-01],
       [ 6.71904452e-03,  5.78021433e-02, -6.27033875e-01, ...,
        -1.93649706e+00,  6.84943973e-01,  6.90703931e-01],
       ...,
       [-2.32629993e-02,  2.36718806e-04,  2.15072314e+00, ...,
         1.11193146e+00, -1.76084389e+00, -1.76257602e+00],
       [ 4.66951030e-02,  6.35572855e-02,  2.14796103e+00, ...,
         1.11193146e+00, -1.76084389e+00, -1.76257602e+00],
       [ 7.66771468e-02,  1.15367568e-01,  2.15072314e+00, ...,
         1.11193146e+00, -1.76084389e+00, -1.76257602e+00]])

In [16]:
# compare the number of repeats for repeated k-fold cross-validation
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot

In [ ]:
# evaluate a model with a given number of repeats
clf = SVC(probability=True)
def evaluate_model_acc(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 
def evaluate_model_apr(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='average_precision', cv=cv, n_jobs=-1)
    return scores

def evaluate_model_f1(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='f1', cv=cv, n_jobs=-1)
    return scores

def evaluate_model_recall(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='recall', cv=cv, n_jobs=-1)
    return scores

# configurations to test
repeats = range(1,4)
results_acc = list()
results_apr = list()
results_f1 = list()
results_recall = list()

for r in repeats:
    # evaluate using a given number of repeats
    scores_acc = evaluate_model_acc(Xs, y, r)
    scores_apr = evaluate_model_apr(Xs, y, r)
    scores_f1 = evaluate_model_f1(Xs, y, r)
    scores_recall = evaluate_model_recall(Xs, y, r)
    # summarize
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_acc), sem(scores_acc), std(scores_acc)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_apr), sem(scores_apr), std(scores_apr)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_f1), sem(scores_f1), std(scores_f1)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_recall), sem(scores_recall),std(scores_recall)))
    # store
    results_acc.append(scores_acc)
    results_apr.append(scores_apr)
    results_f1.append(scores_f1)
    results_recall.append(scores_recall)
# plot the results
pyplot.boxplot(results_acc, labels=[str(r) for r in repeats], showmeans=True)
pyplot.show()

In [17]:
clf = SVC(probability=True)
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, random_state=2, stratify=y)
y_pred = clf.fit(X_train, y_train).predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
cm

array([[466, 465],
       [141, 998]])

In [18]:
print(classification_report(y_test, y_pred ))

              precision    recall  f1-score   support

           0       0.77      0.50      0.61       931
           1       0.68      0.88      0.77      1139

    accuracy                           0.71      2070
   macro avg       0.72      0.69      0.69      2070
weighted avg       0.72      0.71      0.69      2070

